In [1]:
import pickle
import numpy as np

In [31]:
outfile = "output.pkl"
with open(outfile, "rb") as f:
    loaded_dict = pickle.load(f)

In [36]:
n = 37
toks = loaded_dict[n]["sent"]
# for p in loaded_dict[n]["pred"]:
#     print(toks[p])

# args = loaded_dict[n]["args"]
# for a in args:
#     print(a)
#     for i in args[a]:
#         print(toks[i])

res = " ".join(toks)
res

'A man and two children were found dead this morning in a home at Fort Hood , a US Army base in Texas .'

In [ ]:
pred = loaded_dict[0]["pred"]
args = loaded_dict[0]["args"]
print(pred, args)
for tok in loaded_dict[0]["sent"]:
    print(tok)

In [64]:
fn = "lsoie_dev_sample.txt"
with open(fn, "w") as f:
    iter = 0
    last_entry = ""
    for i in range(len(loaded_dict)):
        entry = " ".join(loaded_dict[i]["sent"])
        if entry == last_entry:
            continue
        f.write(f"{entry}\n")
        iter += 1
        last_entry = entry
        if iter == 50:
            break

In [14]:
from graphbrain.hyperedge import Atom, hedge
from graphbrain.learner.pattern_ops import *
from graphbrain.parsers import *
from graphbrain.patterns import PatternCounter
from graphbrain.utils.conjunctions import conjunctions_decomposition

In [27]:
text = """((var posted/Pd.xsxxx.<f-----/en REL) (var (on/Tt/en friday/Cp.s/en) ARG1) (var clark/Cp.s/en ARG0) (var (to/T/en facebook/Cp.s/en) ARG2)
(var ((to/Mi/en explain/Pd.o.-i-----/en) ('s/Bp.am/en clark/Cp.s/en decision/Cc.s/en)) ARG3) (saying/Pd.r.|pg----/en (but/J/en ((usually/M/en 
(do/Mv.|f-----/en (n’t/Mn/en post/P.so.-i-----/en))) (var clark/Cp.s/en ARG0) (political/Ma/en items/Cc.p/en)) (is/Pd.sc.|f--3s-/en 
today/Cc.s/en different/Ca/en))))"""

text2 = """((can/Mm/en (not/Mn/en require/Pd.xsoox.-i-----/en)) (according/Tv.|pg----/en (to/Mt/en (+/B.am/. fox/Cp.s/en news/Cp.s/en))) (the/Md/en 
(+/B.am/. us/Cp.s/en president/Cp.s/en)) (var (+/B.am/. us/Cp.s/en flag/Cc.s/en) ARG0) (var lowering/Cc.s/en REL) (at/T/en (the/Md/en 
(local/Ma/en level/Cc.s/en))))"""

pc = PatternCounter(
    expansions={
        "(* * *)",
        "(* * * *)",
    },
    match_roots={"+/B"},
    count_subedges=True,
)

pc.count(hedge(text))
pc.count(hedge(text2))

print(pc.patterns.most_common(5))

[((+/B.ma */C */C), 3), ((*/B.ma */C */C), 1), ((*/J */R.so */R.sc), 1), ((*/J (*/P.so */C */C) */R.sc), 1), ((*/J */R.so (*/P.sc */C */C)), 1)]


In [ ]:
words_to_i_org = defaultdict(set)
for i, word in enumerate(all_toks_org):
    words_to_i_org[word.lower()].add(i)
print(words_to_i_org)

words_to_i = defaultdict(set)
for i, word in enumerate(all_toks):
    words_to_i[word.lower()].add(i)
print(words_to_i)

for k1, k2 in zip(words_to_i_org.keys(), words_to_i.keys()):
    print(k1, k2)
    old = words_to_i_org[k1]
    new = words_to_i[k2]
    if old == new or old.issubset(new):
        continue

if len(all_toks_org) != len(all_toks):
    print("positions might have changed")
    newpred = set()
    newargs = defaultdict(list)
    for key in words_to_i_org:
        print(key)
        new_pos = words_to_i[key]
        if key in predwords:
            newpred.update(new_pos)
        elif key in argwords.keys():
            label = argwords[key]
            print(label)
            print(new_pos)
            newargs[label].append(new_pos)
        else:
            continue

    newpred = tuple(newpred)
    print(newpred)
    args_sort = dict(sorted(newargs.items()))
    newargs = []
    for label in args_sort:
        newargs.append(tuple(argsdict[label]))
    print(newargs)

    print(pred, args)
    # newargs =
    # for key in words_to_i_org:
    #     print(key)
    #     old = words_to_i_org[key]
    #     if key in words_to_i:
    #         new = words_to_i[key]
    #         # check for equality
    #         if old == new or old.issubset(new):
    #             continue
    #         else:
    #             print(old, new)
    #             if key in predwords:

    #                 print("problem")
    #             elif key in argwords:
    #                 print("problem")
    #             else:
    #                 continue